In [ ]:
import yfinance as yf
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def get_news_sentiment(ticker: str) -> dict:

    stock = yf.Ticker(ticker.upper())
    news = stock.news[:10] if stock.news else []

    sentiment_score = 0
    scored_count = 0  # ✅ initialized before loop

    for article in news:
        content = article.get('content', {})
        title = (
            content.get('title', '')
            or article.get('title', '')
            or ''
        )
        summary = (
            content.get('summary', '')
            or article.get('summary', '')
            or ''
        )
        text = f"{title}. {summary}".strip()
        if text:
            scores = analyzer.polarity_scores(text)
            if abs(scores['compound']) > 0.05:
                sentiment_score += scores['compound']
                scored_count += 1

    avg_score = sentiment_score / scored_count if scored_count else 0
    sentiment = "positive" if avg_score > 0.15 else "negative" if avg_score < -0.15 else "neutral"
    return {
                "ticker": ticker.upper(),
                "sentiment": sentiment,
                "sentiment_score": round(sentiment_score, 4),
                "avg_score": round(avg_score, 4),
                "news_count": len(news),
                "scored_articles": scored_count,  # ✅ useful for debugging
                "recent_headlines": [
                    article.get('content', {}).get('title', '') or article.get('title', '')
                    for article in news[:3]
                ]
            }

